# NVIDIA Usage

## https://blog.langchain.dev/nvidia-nim/

In [1]:
from chatbot import load_literature_db
persist_directory     = "/nfs/turbo/umms-indikar/shared/projects/RAG/databases/Transcription-Factors-5-10-2024/"
ragvectordb, embeddings_model = load_literature_db(persist_directory)

ModuleNotFoundError: No module named 'chatbot'

In [2]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA
from langchain_community.document_loaders import WebBaseLoader
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.output_parsers import StrOutputParser

In [3]:
retriever = ragvectordb.as_retriever()

In [4]:
model = ChatNVIDIA(model="mistral_7b",
                    api_key = "XXXXX")

In [5]:
hyde_template = """Even if you do not know the full answer, generate a one-paragraph hypothetical answer to the below question:

{question}"""
hyde_prompt = ChatPromptTemplate.from_template(hyde_template)
hyde_query_transformer = hyde_prompt | model | StrOutputParser()

In [6]:
from langchain_core.runnables import chain

@chain
def hyde_retriever(question):
    hypothetical_document = hyde_query_transformer.invoke({"question": question})
    return retriever.invoke(hypothetical_document)

In [7]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
answer_chain = prompt | model | StrOutputParser()

In [8]:
@chain
def final_chain(question):
    documents = hyde_retriever.invoke(question)
    for s in answer_chain.stream({"question": question, "context": documents}):
        yield s

In [9]:
for s in final_chain.stream("how can transcription factors help reprogramming?"):
    print(s, end="")

 Transcription factors (TFs) play a crucial role in the process of reprogramming somatic cells into pluripotent stem cells (PSCs), also known as induced pluripotent stem cells (iPSCs). By introducing specific TFs, such as Oct3/4, Sox2, Klf4, and c-Myc, researchers have been able to trigger endogenous expression of pluripotent factors, resulting in the reprogramming of fibroblasts into PSCs. These TFs have been found to be sufficient for converting fibroblasts into iPSCs, and while alternative sets of TFs have been reported for iPSC reprogramming, most studies include Oct3/4 and Sox2. These TFs help erase the memories of somatic cells, enabling them to adopt a pluripotent state. The use of TFs for this purpose has revolutionized the field of regenerative medicine.

In [10]:
for s in final_chain.stream("What some databases about Transcription Factors?"):
    print(s, end="")

 Based on the context provided, there are no specific mentions of databases about Transcription Factors. However, the text does discuss the importance of continued effort to understand TF-mediated gene regulation and mentions two previous comprehensive reviews of human TFs published in 2009. These reviews may contain or be associated with databases of human TFs and binding motifs. The text also mentions the limitations of existing data and the challenges of systematically detecting higher-order interactions, which suggests that there may be ongoing efforts to develop and improve databases in this area.

In [11]:
for s in final_chain.stream("What some databases about Transcription Factors? Which papers are you referencing to answer this question?"):
    print(s, end="")

 The text mentions several databases related to Transcription Factors (TFs) that have been used to study TF-DNA interactions: TRANSFAC, JASPAR, HT-SELEX, CisBP, Interpro, and TFClass. These databases provide information on TF motifs and genomic binding sites. The papers cited in the text for this information are: Matys et al. (2006), Mathelier et al. (2016), Weirauch et al. (2014), Wingender et al. (2015), and Berman et al. (2000).

In [12]:
for s in final_chain.stream("What some databases about Transcription Factors? Provide the title and authors of the papers used to answer this question?"):
    print(s, end="")

 The context does not directly provide information about specific databases related to Transcription Factors. The text discusses papers that have reviewed human Transcription Factors and compiled lists of them, but it does not mention any specific databases.

The papers cited in the context are "The Human Transcription Factors" by Fulton et al., 2009 and "Human Transcription Factors: A Comprehensive Catalogue and Profile-based Analysis" by Vaquerizas et al., 2009.

## Guessing Game with Memory

In [1]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA
from langchain.prompts import PromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

In [2]:
model = ChatNVIDIA(model="mistral_7b",
                   api_key = "XXXXX",
                   stop=['\n', "\n", "Human", "Stop"])

/home/jpic/.local/lib/python3.11/site-packages/langchain_nvidia_ai_endpoints/_statics.py:313: UserWarning: Model mistral_7b is deprecated. Using mistralai/mistral-7b-instruct-v0.2 instead.
  warnings.warn(


In [19]:
template = """The following is a guessing game between you (BRAD) and a human. You guess an integer X between an upper and lower bound and the human will give information of higher > X, lower < X, or = X until you guess correctly. Each turn, use information from the human to update one of either your upper or lower bound for X. By updating your bounds over time, you will narrow the options for X.

Use a binary search strategy. For instance, if we know X is between 0 and 10, guess in the middle 5, or if X is in between 5 and 10, guess in the middle 8.

At each step, answer with your upper bound for X, your lower bound for X, and your guess for X.

You want to use the lowest upper bound and the highest lower bound. These bounds can be found in the chat history where z < x or x > z indicates z is a lower bound for x and x is an upper bound for z.

**Important:** Only provide your output in the specified format below. Do not continue the conversation beyond this format.

Format your response as:

"The upper bound is (Y) and lower bound is (Z). My guess is (X) and (Z) < (X) < (Y). Stop"

Do not continue generating any code after the word "Stop". There will be multiple iterations but you only create the response one time per iteration.

Example:

Iteration 0:
Human: Guess a number between 0 and 20
System: The upper bound is 20 and lower bound is 0. My guess is 8 and 0 < 8 < 20

Iteration 1:
Human: the number is > 8
System: The upper bound is 20 and lower bound is 8. My guess is 15 and 8 < 15 < 20

Iteration 2
Human: the number is < 15
System: The upper bound is 15 and lower bound is 8. My guess is 12 and 8 < 12 < 15
END EXAMPLE. The numbers above are just for a workflow. Do not base your guesses on these

Current conversation:
{history}

Human: {input}
BRAD:"""

# Ensure PROMPT object uses the new template
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
memory = ConversationBufferMemory(ai_prefix="BRAD")

conversation = ConversationChain(
    prompt=PROMPT,
    llm=model,
    verbose=True,
    memory=memory,
)

In [20]:
user_input = "Guess a number with a lower bound of 0 and upper bound 100."
response = conversation.predict(input=user_input)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a guessing game between you (BRAD) and a human. You guess an integer X between an upper and lower bound and the human will give information of higher > X, lower < X, or = X until you guess correctly. Each turn, use information from the human to update one of either your upper or lower bound for X. By updating your bounds over time, you will narrow the options for X.

Use a binary search strategy. For instance, if we know X is between 0 and 10, guess in the middle 5, or if X is in between 5 and 10, guess in the middle 8.

At each step, answer with your upper bound for X, your lower bound for X, and your guess for X.

You want to use the lowest upper bound and the highest lower bound. These bounds can be found in the chat history where z < x or x > z indicates z is a lower bound for x and x is an upper bound for z.

**Important:** Only provide your output in the specified format below. Do not continue t

In [21]:
user_input = "The number is < 50. Give the next iteration and stop the response at the word Stop"
response = conversation.predict(input=user_input)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a guessing game between you (BRAD) and a human. You guess an integer X between an upper and lower bound and the human will give information of higher > X, lower < X, or = X until you guess correctly. Each turn, use information from the human to update one of either your upper or lower bound for X. By updating your bounds over time, you will narrow the options for X.

Use a binary search strategy. For instance, if we know X is between 0 and 10, guess in the middle 5, or if X is in between 5 and 10, guess in the middle 8.

At each step, answer with your upper bound for X, your lower bound for X, and your guess for X.

You want to use the lowest upper bound and the highest lower bound. These bounds can be found in the chat history where z < x or x > z indicates z is a lower bound for x and x is an upper bound for z.

**Important:** Only provide your output in the specified format below. Do not continue t

In [22]:
user_input = "The number is > 25. Give the next iteration and stop the response at the word Stop"
response = conversation.predict(input=user_input)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a guessing game between you (BRAD) and a human. You guess an integer X between an upper and lower bound and the human will give information of higher > X, lower < X, or = X until you guess correctly. Each turn, use information from the human to update one of either your upper or lower bound for X. By updating your bounds over time, you will narrow the options for X.

Use a binary search strategy. For instance, if we know X is between 0 and 10, guess in the middle 5, or if X is in between 5 and 10, guess in the middle 8.

At each step, answer with your upper bound for X, your lower bound for X, and your guess for X.

You want to use the lowest upper bound and the highest lower bound. These bounds can be found in the chat history where z < x or x > z indicates z is a lower bound for x and x is an upper bound for z.

**Important:** Only provide your output in the specified format below. Do not continue t

In [23]:
user_input = "The number is > 38. Give the next iteration and stop the response at the word Stop"
response = conversation.predict(input=user_input)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a guessing game between you (BRAD) and a human. You guess an integer X between an upper and lower bound and the human will give information of higher > X, lower < X, or = X until you guess correctly. Each turn, use information from the human to update one of either your upper or lower bound for X. By updating your bounds over time, you will narrow the options for X.

Use a binary search strategy. For instance, if we know X is between 0 and 10, guess in the middle 5, or if X is in between 5 and 10, guess in the middle 8.

At each step, answer with your upper bound for X, your lower bound for X, and your guess for X.

You want to use the lowest upper bound and the highest lower bound. These bounds can be found in the chat history where z < x or x > z indicates z is a lower bound for x and x is an upper bound for z.

**Important:** Only provide your output in the specified format below. Do not continue t

In [24]:
user_input = "The number is < 45. Give the next iteration and stop the response at the word Stop"
response = conversation.predict(input=user_input)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a guessing game between you (BRAD) and a human. You guess an integer X between an upper and lower bound and the human will give information of higher > X, lower < X, or = X until you guess correctly. Each turn, use information from the human to update one of either your upper or lower bound for X. By updating your bounds over time, you will narrow the options for X.

Use a binary search strategy. For instance, if we know X is between 0 and 10, guess in the middle 5, or if X is in between 5 and 10, guess in the middle 8.

At each step, answer with your upper bound for X, your lower bound for X, and your guess for X.

You want to use the lowest upper bound and the highest lower bound. These bounds can be found in the chat history where z < x or x > z indicates z is a lower bound for x and x is an upper bound for z.

**Important:** Only provide your output in the specified format below. Do not continue t

In [25]:
user_input = "The number is < 42. Give the next iteration and stop the response at the word Stop"
response = conversation.predict(input=user_input)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a guessing game between you (BRAD) and a human. You guess an integer X between an upper and lower bound and the human will give information of higher > X, lower < X, or = X until you guess correctly. Each turn, use information from the human to update one of either your upper or lower bound for X. By updating your bounds over time, you will narrow the options for X.

Use a binary search strategy. For instance, if we know X is between 0 and 10, guess in the middle 5, or if X is in between 5 and 10, guess in the middle 8.

At each step, answer with your upper bound for X, your lower bound for X, and your guess for X.

You want to use the lowest upper bound and the highest lower bound. These bounds can be found in the chat history where z < x or x > z indicates z is a lower bound for x and x is an upper bound for z.

**Important:** Only provide your output in the specified format below. Do not continue t

In [26]:
user_input = "The number is > 40. Give the next iteration and stop the response at the word Stop"
response = conversation.predict(input=user_input)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a guessing game between you (BRAD) and a human. You guess an integer X between an upper and lower bound and the human will give information of higher > X, lower < X, or = X until you guess correctly. Each turn, use information from the human to update one of either your upper or lower bound for X. By updating your bounds over time, you will narrow the options for X.

Use a binary search strategy. For instance, if we know X is between 0 and 10, guess in the middle 5, or if X is in between 5 and 10, guess in the middle 8.

At each step, answer with your upper bound for X, your lower bound for X, and your guess for X.

You want to use the lowest upper bound and the highest lower bound. These bounds can be found in the chat history where z < x or x > z indicates z is a lower bound for x and x is an upper bound for z.

**Important:** Only provide your output in the specified format below. Do not continue t

In [27]:
user_input = "The number is < 43. Give the next iteration and stop the response at the word Stop"
response = conversation.predict(input=user_input)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a guessing game between you (BRAD) and a human. You guess an integer X between an upper and lower bound and the human will give information of higher > X, lower < X, or = X until you guess correctly. Each turn, use information from the human to update one of either your upper or lower bound for X. By updating your bounds over time, you will narrow the options for X.

Use a binary search strategy. For instance, if we know X is between 0 and 10, guess in the middle 5, or if X is in between 5 and 10, guess in the middle 8.

At each step, answer with your upper bound for X, your lower bound for X, and your guess for X.

You want to use the lowest upper bound and the highest lower bound. These bounds can be found in the chat history where z < x or x > z indicates z is a lower bound for x and x is an upper bound for z.

**Important:** Only provide your output in the specified format below. Do not continue t

## NVIDIA with BRAD

### LLM + Searching for New Literature

In [19]:
from BRAD import llms
from langchain import PromptTemplate, LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

llm = llms.load_nvidia()
memory = ConversationBufferMemory(ai_prefix="BRAD")

template = """Current conversation:\n{history}

Query:{input}

From the query, decide if ARXIV, PUBMED, or BIORXIV should be searched, and propose no more than 10 words for this query and database.

Database: <ARXIV, PUBMED, or BIORXIV>
Search Terms: <improved search terms>
"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = ConversationChain(prompt  = PROMPT,
                                 llm     = llm,
                                 verbose = True,
                                 memory  = memory,
                                )

/home/jpic/.local/lib/python3.11/site-packages/langchain_nvidia_ai_endpoints/_statics.py:313: UserWarning: Model mistral_7b is deprecated. Using mistralai/mistral-7b-instruct-v0.2 instead.
  warnings.warn(


In [20]:
prompt = "What is the latest work in transcription factor control of the cell cycle?"
response = conversation.predict(input=prompt)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
Current conversation:


Query:What is the latest work in transcription factor control of the cell cycle?

From the query, decide if ARXIV, PUBMED, or BIORXIV should be searched, and propose no more than 10 words for this query and database.

Database: <ARXIV, PUBMED, or BIORXIV>
Search Terms: <improved search terms>


> Finished chain.
 Database: PUBMED
Search Terms: Transcription factors, Cell cycle regulation, Latest findings.

You can also add specific transcription factors or cell types for more precise results. For example, "Myc transcription factor and cell cycle regulation" or "Transcription factor role in G2 checkpoint in human cells".


In [21]:
prompt = "list the most recent unpublished work on BIORXIV for reprogramming"
response = conversation.predict(input=prompt)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
Current conversation:
Human: What is the latest work in transcription factor control of the cell cycle?
BRAD:  Database: PUBMED
Search Terms: Transcription factors, Cell cycle regulation, Latest findings.

You can also add specific transcription factors or cell types for more precise results. For example, "Myc transcription factor and cell cycle regulation" or "Transcription factor role in G2 checkpoint in human cells".

Query:list the most recent unpublished work on BIORXIV for reprogramming

From the query, decide if ARXIV, PUBMED, or BIORXIV should be searched, and propose no more than 10 words for this query and database.

Database: <ARXIV, PUBMED, or BIORXIV>
Search Terms: <improved search terms>


> Finished chain.
 Database: BIORXIV
Search Terms: Reprogramming, Latest findings, Unpublished work.


In [22]:
prompt = "What is the latest work in bifurcation detection on manifolds?"
response = conversation.predict(input=prompt)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
Current conversation:
Human: What is the latest work in transcription factor control of the cell cycle?
BRAD:  Database: PUBMED
Search Terms: Transcription factors, Cell cycle regulation, Latest findings.

You can also add specific transcription factors or cell types for more precise results. For example, "Myc transcription factor and cell cycle regulation" or "Transcription factor role in G2 checkpoint in human cells".
Human: list the most recent unpublished work on BIORXIV for reprogramming
BRAD:  Database: BIORXIV
Search Terms: Reprogramming, Latest findings, Unpublished work.

Query:What is the latest work in bifurcation detection on manifolds?

From the query, decide if ARXIV, PUBMED, or BIORXIV should be searched, and propose no more than 10 words for this query and database.

Database: <ARXIV, PUBMED, or BIORXIV>
Search Terms: <improved search terms>


> Finished chain.
 Database: ARXIV
Search Terms: Bifurcatio

In [23]:
def parse_llm_response(response):
    """
    Parses the LLM response to extract the database name and search terms.
    
    Parameters:
    response (str): The response from the LLM.
    
    Returns:
    dict: A dictionary with the database name and a list of search terms.
    """
    # Initialize an empty dictionary to hold the parsed data
    parsed_data = {}

    # Split the response into lines
    lines = response.strip().split('\n')

    # Extract the database name
    database_line = lines[0].replace("Database:", "").strip()
    parsed_data["database"] = database_line

    # Extract the search terms
    search_terms_line = lines[1].replace("Search Terms:", "").strip()
    search_terms = [term.strip() for term in search_terms_line.split(',')]
    parsed_data["search_terms"] = search_terms

    return parsed_data

# Example LLM response
llm_response = """
Database: ARXIV
Search Terms: Bifurcation detection, Manifolds, Latest preprints
"""

# Parse the LLM response
parsed_response = parse_llm_response(llm_response)
print(parsed_response)


{'database': 'ARXIV', 'search_terms': ['Bifurcation detection', 'Manifolds', 'Latest preprints']}


### LLM + Gene Ontology + Enrichr

In [1]:
from BRAD import llms
llm = llms.load_nvidia()

[nltk_data] Downloading package words to /home/jpic/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jpic/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jpic/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Enter your NVIDIA API key:  ········


/home/jpic/.local/lib/python3.11/site-packages/langchain_nvidia_ai_endpoints/_statics.py:313: UserWarning: Model mistral_7b is deprecated. Using mistralai/mistral-7b-instruct-v0.2 instead.
  warnings.warn(


In [2]:
from langchain import PromptTemplate, LLMChain

In [3]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(ai_prefix="BRAD")

In [8]:
from langchain.chains import ConversationChain
template = """Current conversation:\n{history}\n\n\nNew You are an expert in bioinformatics. A user has provided the following query related to their dataset. Query:\n{input}

Please determine which database (Enrichr or Gene Ontology) is most suitable for this query and list the relevant genes from the dataset. Provide your response in the following format:

Database: [Database name]
Relevant Genes: [List of relevant genes]
"""
PROMPT = PromptTemplate(input_variables=["history", "input"], template=template)
conversation = ConversationChain(prompt  = PROMPT,
                                 llm     = llm,
                                 verbose = True,
                                 memory  = memory,
                                )

In [9]:
prompt = "Look up MYOD, MYC, cMYC, and PCNA in gene ontology"
response = conversation.predict(input=prompt)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
Current conversation:
Human: Look up MYOD, MYC, cMYC, and PCNA in gene ontology
BRAD:  I'd be happy to help you look up information about the genes MYOD, MYC, cMYC, and PCNA in the Gene Ontology (GO). The Gene Ontology is a standardized database used to unify the representation of gene and gene product functions across all organisms.

Let's look up each gene one by one to understand their functions in the context of the biological process, molecular function, and cellular component categories in the Gene Ontology:

1. MYOD:
Myod is a gene that encodes the myoblast determination protein 1 (MYOD1). MYOD1 is a transcription factor that plays a critical role in the differentiation of muscle cells. In the Gene Ontology, MYOD1 is associated with several biological processes such as muscle cell differentiation, muscle cell development, and muscle cell proliferation. At the molecular function level, it interacts with DNA as a

In [4]:
from langchain.chains import ConversationChain
PROMPT = PromptTemplate(
    input_variables=['user_query', 'history'],
    template="""
    History: {history}
    
    You are an expert in bioinformatics. A user has provided the following query related to their dataset:

    Query: {user_query}

    Please determine which database (Enrichr or Gene Ontology) is most suitable for this query and list the relevant genes from the dataset. Provide your response in the following format:

    Database: [Database name]
    Relevant Genes: [List of relevant genes]
    """
)
conversation = ConversationChain(prompt  = PROMPT,
                                 llm     = llm,
                                 verbose = True,
                                 memory  = memory
                                )
response = conversation.predict(input=prompt)

ValidationError: 1 validation error for ConversationChain
__root__
  Got unexpected prompt input variables. The prompt expects ['history', 'user_query'], but got ['history'] as inputs from memory, and input as the normal input key. (type=value_error)

In [5]:
from langchain import PromptTemplate, LLMChain

prompt_template = PromptTemplate(
    input_variables=["user_query"],
    template="""
    You are an expert in bioinformatics. A user has provided the following query related to their dataset:

    Query: {user_query}

    Please determine which database (Enrichr or Gene Ontology) is most suitable for this query and list the relevant genes from the dataset. Provide your response in the following format:

    Database: [Database name]
    Relevant Genes: [List of relevant genes]
    """
)

In [6]:
llm_chain = LLMChain(
    prompt=prompt_template,
    llm=llm
)

ValidationError: 1 validation error for LLMChain
llm
  Can't instantiate abstract class BaseLanguageModel with abstract methods agenerate_prompt, apredict, apredict_messages, generate_prompt, predict, predict_messages (type=type_error)

In [7]:
type(llm)

langchain_nvidia_ai_endpoints.chat_models.ChatNVIDIA

### Integration

In [1]:
import sys
sys.path.append('/home/jpic/RAG-DEV/BRAD/')

In [9]:
import importlib
import brad
importlib.reload(brad)
from brad import *

In [10]:
model_path            = '/nfs/turbo/umms-indikar/shared/projects/RAG/models/llama-2-7b-chat.Q2_K.gguf' # llama-2-7b-chat.Q8_0.gguf'
persist_directory     = "/nfs/turbo/umms-indikar/shared/projects/RAG/databases/Transcription-Factors-5-10-2024/"
llm, callback_manager = load_llama(nvidia_api_key = "XXXXX",
                                   nvidia_model   = "mistral_7b",
                                  )
ragvectordb, embeddings_model = load_literature_db(persist_directory)

Sun Jun  2 17:32:27 2024 INFO Use pytorch device_name: cpu
Sun Jun  2 17:32:27 2024 INFO Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
/home/jpic/.local/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Sun Jun  2 17:32:28 2024 INFO Collection DB_cosine_cSize_700_cOver_200 is not created.


In [11]:
main(llm=llm, ragvectordb=ragvectordb, embeddings_model=embeddings_model)

Sun Jun  2 17:32:30 2024 INFO local


Welcome to RAG! The chat log from this conversation will be saved to logs/RAG2024-06-02-17:32:30.549192.json. How can I help?


Input >>  what role do transcription factors play in cell reprogramming?


Sun Jun  2 17:32:51 2024 INFO RAG


RAG >> 1: 

...

Input >>  list the most important transcription factors for reprogramming


Sun Jun  2 17:33:59 2024 INFO RAG


RAG >> 3:  The reprogramming factors are a subset of eukaryotic transcription factors that have the ability to reprogram one type of cell into another. These factors are crucial for initial cell programming in embryonic development. examples of such factors include MYOD for fibroblast to myoblast conversion, GATA1 for myeloblast, and CEBP for B lymphocyte. The identification of these candidate reprogramming factors was based on the fact that genes that contribute to the identities of fertilized eggs or ES cells were thought to be responsible for the reconstitution of specific transcriptional networks during reprogramming. The seminal work leading to the discovery of these factors and their use for the generation of the first iPPS (induced pluripotent stem cells) was described in the article "Induction of pluripotency: a status report" by Soufi et al. (2006). Other references, such as Spitz and Furlong (2012), Stormo and Zhao (2010), and Stubbs et al. (2011), provide additional informat

 Y


NameError: name 'gonto' is not defined

In [2]:
help(load_llama)

Help on function load_llama in module chatbot:

load_llama(model_path='/nfs/turbo/umms-indikar/shared/projects/RAG/models/llama-2-7b-chat.Q8_0.gguf', nvidia_api_key=None, nvidia_model=None)



### Example Usage

In [4]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings, ChatNVIDIA
import importlib
import chatbot
importlib.reload(chatbot)
from chatbot import *

In [2]:
persist_directory     = "/nfs/turbo/umms-indikar/shared/projects/RAG/databases/Transcription-Factors-5-10-2024/"
ragvectordb, embeddings_model = load_literature_db(persist_directory)

Sat 01 Jun 2024 02:38:03 PM EDT INFO Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Sat 01 Jun 2024 02:38:04 PM EDT INFO Use pytorch device_name: cpu
Sat 01 Jun 2024 02:38:04 PM EDT INFO Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
Sat 01 Jun 2024 02:38:04 PM EDT INFO Collection DB_cosine_cSize_700_cOver_200 is not created.


In [5]:
model = ChatNVIDIA(model   = "mistral_7b",
                   api_key = "XXXXX",
                  )
#                   stop=['\n', "\n", "Human", "Stop"])

In [7]:
prompt = "how can transcription factors help reprogramming?"

In [8]:
# query to database
documentSearch = ragvectordb.similarity_search_with_relevance_scores(prompt)
docs, scores = getDocumentSimilarity(documentSearch)

# pass the database output to the llm
chain = load_qa_chain(model, chain_type="stuff")
res = chain({"input_documents": docs, "question": prompt})


/home/jpic/.local/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [11]:
res['output_text']

' Transcription factors are crucial for the initial cell programming during embryonic development and play a key role in cell reprogramming by engaging genes that are developmentally silenced. In the context of the provided text, the specific transcription factors identified for their ability to reprogram somatic cells into pluripotent stem cells include OCT4, SOX2, and other factors. These factors were discovered through seminal work that led to the generation of the first induced pluripotent stem cells (iPSCs), as described in the article. The process of identifying these factors was based on the assumption that genes responsible for the identities of fertilized eggs or ES cells would contribute to the reconstitution of specific transcriptional networks during reprogramming.'